# YOLO モデルを再トレーニングする

YOLO モデルを再トレーニングするには、中程度の事故と重大な事故のラベルが付いた自動車画像の準備されたデータセットが必要です。  このようなデータセット（RoboFlow から取得） には、トレーニング データセットと検証データセットに分割された注釈付き画像がすでに含まれています。  これらのトレーニング/検証セットを使用して、現在の YOLO モデルを再トレーニングします。トレーニングに必要なデータ構造に関するいくつかの情報を以下に示します。

1. モデルに検出を教えたいオブジェクトのエンコード クラスは、0-`moderate` と 1-`severe` です。
2. データセットはフォルダー内にあり、その中に 2 つのサブフォルダー（`train` と `valid`） があります。各サブフォルダー内には、 `images` と `labels` の 2 つのサブフォルダーがあります。
3. 各画像には、 `labels` サブフォルダー内に対応する注釈テキスト ファイルがあります。注釈テキスト ファイルには、画像ファイルと同じ名前が付けられます。
4. データセット記述子 YAML ファイル（data.yaml） はデータセットを指し、データセット内のオブジェクト クラスを記述します。この YAML ファイルはモデルの `train` メソッドに渡されて、トレーニング プロセスを開始します。

さぁ、始めましょう!

In [ ]:
# このラボ用に設計されたワークベンチ イメージを使用しなかった場合は、コメントを解除して次の行を実行して、必要なパッケージをインストールできます。
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

import os
import requests
import zipfile
from tqdm.notebook import tqdm
from ultralytics import YOLO

次に、YOLO モデル 'yolo8m.pt' をロードします。

In [ ]:
# モデルをロードします。
model = YOLO('yolov8m.pt')  # 事前トレーニング済みモデルをロードします （トレーニングに推奨）

## トレーニングデータを取得する

次の 2 つのトレーニング データ セットが zip ファイルとして提供されています。
1) `accident-full.zip`   - モデルを完全に再トレーニングする場合に使用します。
2) `accident-sample.zip` - モデルを完全に再トレーニングする時間がない場合に、モデルを部分的に再トレーニングするために使用します。

このワークショップではサンプル データセットを使用します。

In [ ]:
# データセットを取得します。
def retrieve_dataset(dataset_type):

    # ディレクトリが存在するかどうかを確認し、存在しない場合は作成します。
    if not os.path.exists("./datasets/"):
        os.makedirs("./datasets/")

    URL = f"https://rhods-public.s3.amazonaws.com/sample-data/accident-data/accident-{dataset_type}.zip"

    # ファイルが存在するかどうかを確認し、存在しない場合はダウンロードして解凍します。
    if not os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Downloading file...")
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        t = tqdm(total=total_size, unit='iB', unit_scale=True)
        with open(f'./datasets/accident-{dataset_type}.zip', 'wb') as f:
            for data in response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    if os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Unzipping file...")
        with zipfile.ZipFile(f'./datasets/accident-{dataset_type}.zip', 'r') as zip_ref:
            zip_ref.extractall(path='./datasets/')
    print("Done!")


dataset_type = 'sample'
# dataset_type = 'full' # 完全なデータセットを取得する場合は、代わりにこの行を使用してください。
retrieve_dataset(dataset_type)

## YOLO モデルの再トレーニング

 'epoch' とは何かを理解することから始めましょう。  機械学習モデルは、アルゴリズムを通過した特定のデータセットを使用してトレーニングされます。完全なデータセットがアルゴリズムを通過するたびに、 'epoch' が完了したと言われます。各  'epoch' では、モデルのトレーニングがさらに洗練されます。

以下のトレーニング コードでは、さまざまなパラメーターを設定します。 

**results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)**

- epochs: プロセスをデモするだけなので、epoch は 1 を使用します。
- imgsz: これはモデルに供給する必要がある画像のサイズです。
- batch: これは、アルゴリズムが同時に処理する画像の数です。多ければ多いほど良い結果が得られますが、消費するメモリも多くなります。このワークショップでは限られたリソースを使用しており、これは単なるトレーニング例であるため、意図的に低い値の 2 を使用します。

トレーニングの実行では、各 **epoch<** にトレーニング フェーズと検証フェーズの両方の概要が表示されます。行 1 と 2 はトレーニング フェーズの結果を示し、行 3 と 4 は各エポックの検証フェーズの結果を示します。

次のセルを実行して、モデルの再トレーニングを開始します。

In [ ]:
# モデルをトレーニングします。

results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)

**Note**: 標準的な再トレーニングの後、結果に満足できれば、モデルを ONNX 形式にエクスポートできます。 

（次のコマンドで trainX を使用するトレーニング セッションに置き換えます）

`ObjDetOXModel = YOLO("runs/detect/trainX/weights/best.pt").export(format="onnx")`

## トレーニング結果の解釈

 **full dataset** に対するトレーニングのプロセスと結果の詳細な説明は、ラボガイドで確認できます。

完了したら、ノートブックを閉じて次のページに進むことができます。

モデルを再トレーニングしたので、自動車事故のある画像に対してテストしてみましょう。